#PEFT  LoRa




In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Инициализация

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "IlyaGusev/saiga2_7b_lora",
    load_in_4bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Замаорозка весов

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Настройка Лора адаптера

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3517190144 || trainable%: 0.2385031134671575


## Data

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("dkagramanyan/horoscopes_ru")


Generating train split:   0%|          | 0/66501 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6976 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 66501
    })
    test: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 6976
    })
})

In [ ]:
data['train']['sign']

In [ ]:
def merge_columns(example):
    example["prediction"] = "Гороскоп на завтра: " + str(example["text"][:512])
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

Map:   0%|          | 0/66501 [00:00<?, ? examples/s]

['Гороскоп на завтра: Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.',
 'Гороскоп на завтра: Первый день нового года прекрасно подходит для начинаний, которые вы постоянно откладывали в прошлом году. Отбросьте все сомнения и действуйте. Упор может быть сделан как на повышение профессиональной квалификации, так и на реформацию личной жизни.',
 'Гороскоп на завтра: Этот день как нельзя лучше подходит для улучшения состояния здоровья и пополнения физического потенциала. Посетите тренажерный зал, бассейн или начните делать зарядку по утрам.',
 'Гороскоп на завтра: Первый день нового года, да еще понедельник будет для вас спокойным, днем, когда вы будете сторонним наблюдателем всех событий, происходящих вокруг. Вам не придется лично принимать важные решения.',
 'Гороскоп на завтра: Львы не сорвутся в начале года с места в карьер, они немного у

In [ ]:
data['train'][0]

{'date': '2004-01-01',
 'sign': 'овен',
 'text': 'Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.',
 'prediction': 'Гороскоп на завтра: Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.'}

In [ ]:
data = data['train'].map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [ ]:
data

Dataset({
    features: ['date', 'sign', 'text', 'prediction', 'input_ids', 'attention_mask'],
    num_rows: 66501
})

### Training

In [ ]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
1,2.210600
2,2.255500
3,2.317100
4,2.232900
5,2.226400
6,2.270000
7,2.250700
8,2.294500
9,2.240800
10,2.228200


TrainOutput(global_step=200, training_loss=1.8372541570663452, metrics={'train_runtime': 2606.2785, 'train_samples_per_second': 1.228, 'train_steps_per_second': 0.077, 'total_flos': 2.472217281842381e+16, 'train_loss': 1.8372541570663452, 'epoch': 0.05})

## Загрузка в хаб

In [ ]:
model.push_to_hub("Roflmax/saiga2_7b_lora",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Roflmax/saiga2_7b_lora/commit/8ac59429f539a699cfe091b7ac4b6af10fabdcc1', commit_message='basic training', commit_description='', oid='8ac59429f539a699cfe091b7ac4b6af10fabdcc1', pr_url=None, pr_revision=None, pr_num=None)

## Загрузка с хаба

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Roflmax/saiga2_7b_lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

## Примеры

In [ ]:
batch = tokenizer("", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на завтра: День будет интересен как в общении с подругами, так и в поисках достойного работодателя. Проведение поговорок с други


In [ ]:
batch = tokenizer("", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на завтра: Проживёт вполне приемлемое для Дыса начало дня, причем вам станет легко избавиться от тягот и нес


In [ ]:
batch = tokenizer("Гороскоп на завтра:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на завтра: Успех в личной жизни зависит от того, как и какие намерения вы формулировали. Всплеснувшиеся в прошлый период романтические настроения будут


In [ ]:
batch = tokenizer("Гороскоп на завтра у скорпионов: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на завтра у скорпионов:  Во второй половине дня в сознании появятся очеговаримые соображения против чего-то. В отношениях с близкими к вам людьми, в частности в семье


In [ ]:
batch = tokenizer("Гороскоп на 29 июля: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на 29 июля:  Вам удастся излечить любовную травму, наполнить одиночеством прежних сказочных ситуаций. Близкое тело можно обрести в близки


In [ ]:
batch = tokenizer("Гороскоп на завтра: Ужастный день", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп на завтра: Ужастный день. В течение дня от вас будет требовать немного финансовой компетенции. Активно работает на выгоду либо вам, либо окружающим. Людям,


In [ ]:
batch = tokenizer("Гороскоп - это развод и неправда", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп - это развод и неправда. Ненавина выходит из этого поиска. Действительность размывается и осложняется иллюзиями взаимных намерений. Этот день отличен от обычного.


In [ ]:
batch = tokenizer("Гороскоп - это развод и неправда", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп - это развод и неправда во взглядах на будущее, большой уход из общего житейского плана, полное преображение в чужую форму, выворот отношений с окружающими, смещение с


In [ ]:
batch = tokenizer("Гороскоп - это развод и неправда, но вот предсказание на завтра:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Гороскоп - это развод и неправда, но вот предсказание на завтра: вы будете поняты людьми, прийдет доброжелательный оправотку. Всюду будет достаток. Зато из-за этого может появиться новая нужда:
